# GNN Training

## Imports

In [ ]:
import os

import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data.sampler import SequentialSampler
from torch_geometric.data import DataLoader

from pg_networks.dynamic_edge import DynamicEdge
from pg_networks.gcn import GCN
from pg_networks.sg import SG
import src.config as cfg
from src.davis_2016 import DAVIS2016
from src.solver import Solver
from src.vis_utils import plot_img_with_contour_and_translation, plot_translations, save_loss, \
                          plot_combo_img

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Train and Val Dataset

In [ ]:
train = DAVIS2016(cfg.PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH,
                  cfg.ANNOTATIONS_AUGMENTED_FOLDERS_PATH, cfg.CONTOURS_FOLDERS_PATH, 
                  cfg.IMAGES_AUGMENTED_FOLDERS_PATH, cfg.TRANSLATIONS_FOLDERS_PATH,
                  cfg.PARENT_MODEL_PATH,
                  cfg.LAYER, cfg.K, cfg.AUGMENTATION_COUNT,
                  cfg.SKIP_SEQUENCES, cfg.TRAIN_SEQUENCES[:cfg.NUM_TRAIN_SEQUENCES], cfg.VAL_SEQUENCES[:cfg.NUM_VAL_SEQUENCES],
                  train=True)

In [ ]:
val = DAVIS2016(cfg.PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH,
                cfg.ANNOTATIONS_AUGMENTED_FOLDERS_PATH, cfg.CONTOURS_FOLDERS_PATH, 
                cfg.IMAGES_AUGMENTED_FOLDERS_PATH, cfg.TRANSLATIONS_FOLDERS_PATH,
                cfg.PARENT_MODEL_PATH,
                cfg.LAYER, cfg.K, 0,
                cfg.SKIP_SEQUENCES, cfg.TRAIN_SEQUENCES[:cfg.NUM_TRAIN_SEQUENCES], cfg.VAL_SEQUENCES[:cfg.NUM_VAL_SEQUENCES],
                train=False)

In [ ]:
print("Train size: %i" % len(train))
print("Val size: %i" % len(val))

## Visualize some samples

In [ ]:
num_to_display = 5

for i in range(num_to_display):
    
    fig = plt.figure(figsize=(num_to_display*10,10))
    
    # randomly select a sample
    rand_i = np.random.randint(0, len(train))
    data = train[rand_i]
    
    # Load corresponding image
    processed_file_name = train.processed_file_names[rand_i]
    folder = processed_file_name[:-11]
    augmentation_count = processed_file_name[-10:-9]
    file_name = processed_file_name[-8:-3]
    
    image_path = os.path.join(val.raw_paths[0], folder, augmentation_count,
                                ('{}{}'.format(file_name, '.png')))
    if not os.path.exists(image_path):
        continue
    
    image = imageio.imread(image_path)
    
    ax = plt.subplot(1, num_to_display, i + 1)
    ax.set_title('Sample #{}'.format(rand_i))
    
    plot_img_with_contour_and_translation(image, data.contour, data.y)

## Simple GCN

In [ ]:
data = train[0]

model = GCN(in_channels=data.num_features, 
            out_channels=data.y.shape[1])

print(model)

## Overfit

In [ ]:
num_train = 80
num_val = 20

overfit_train_loader = DataLoader(train, batch_size=16, 
                                  shuffle=False, sampler=SequentialSampler(range(num_train)))
overfit_val_loader = DataLoader(train, batch_size=1, 
                                shuffle=False, sampler=SequentialSampler(range(num_val)))

# Load model and run the solver
overfit_model = GCN(in_channels=data.num_features, 
                    out_channels=data.y.shape[1])

overfit_solver = Solver(optim_args={"lr": cfg.LEARNING_RATE})
overfit_solver.train(overfit_model, overfit_train_loader, overfit_val_loader,
                     num_epochs=5, log_nth=100, verbose=True)

In [ ]:
save_loss(overfit_solver)

## Train

In [ ]:
train_loader = DataLoader(train, batch_size=cfg.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val, batch_size=cfg.BATCH_SIZE, shuffle=False)

# Load model and run the solver
model = GCN(in_channels=train[0].num_features, 
            out_channels=train[0].y.shape[1])

solver = Solver(optim_args={"lr": cfg.LEARNING_RATE,
                            "weight_decay": cfg.WEIGHT_DECAY})

solver.train(model, train_loader, val_loader,
             num_epochs=cfg.NUM_EPOCHS, log_nth=1000, verbose=True)

In [ ]:
save_loss(solver)

## Visualize outputs (ground truth in green, prediction in red)

In [ ]:
model.cpu()
model.double()
model.eval()
num_to_display = 10

for i in range(num_to_display):

    fig = plt.figure(figsize=(num_to_display*10,10))
    
    # randomly select a sample
    rand_i = np.random.randint(0, len(val))
    data = val[rand_i]
    
    # Load corresponding image
    processed_file_name = val.processed_file_names[rand_i]
    folder = processed_file_name[:-11]
    augmentation_count = processed_file_name[-10:-9]
    file_name = processed_file_name[-8:-3]
    
    image_path = os.path.join(val.raw_paths[0], folder, augmentation_count,
                                ('{}{}'.format(file_name, '.png')))
    
    image = imageio.imread(image_path)
    
    with torch.no_grad():
        translation_pred = model(data)
    
    ax = plt.subplot(1, num_to_display, i + 1)
    ax.set_title('Sample #{}'.format(rand_i))
    
    plot_translations(image, data.contour, data.y, translation_pred) 

## Hyperparameter Tuning

In [ ]:
train_loader = DataLoader(train, batch_size=cfg.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val, batch_size=1, shuffle=False)

learning_rates = [1e-1, 1e-2, 1e-3]
weight_decays = [1e1, 0, 1e-0]

best_model = None
best_loss = 1e10

i = 0
for learning_rate in learning_rates:
    for weight_decay in weight_decays:
        model = GCN(in_channels=train[0].num_features, 
                    out_channels=train[0].y.shape[1])
        solver = Solver(optim_args={"lr": learning_rate,
                                    "weight_decay": weight_decay})
        solver.train(model, train_loader, val_loader,
                     num_epochs=10, log_nth=100, verbose=False)

        # Predict on the validation set
        val_loss = max(solver.val_loss_history)
        print('Hyperparamter Tuning #', i + 1,
                'lr: ', learning_rate,
                'wd: ', weight_decay,
                'val_loss: ', val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model

        i += 1